## Model 1 DINO v2 Giant

In [1]:
%%writefile model_1_dino_giant.py
# ====================================================================================
# Model 1: DINOv2-Giant + Lasso Regression Inference Script (CLEAN + FIXED)
# ====================================================================================

import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoImageProcessor, AutoModel
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from PIL import Image
import warnings

warnings.filterwarnings('ignore')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def run_dinov2_lasso_inference(model_id, desc):

    ROOT = "/kaggle/input/csiro-biomass/"

    # ---------------------------------------------------------------
    # 1. Load DINOv2
    # ---------------------------------------------------------------
    processor = AutoImageProcessor.from_pretrained(
        f"/kaggle/input/dinov2/pytorch/{model_id}/1"
    )
    model = AutoModel.from_pretrained(
        f"/kaggle/input/dinov2/pytorch/{model_id}/1"
    ).to(DEVICE)
    model.eval()

    print("  Models loaded.")

    # ---------------------------------------------------------------
    # 2. Load training data
    # ---------------------------------------------------------------
    train_df = pd.read_csv(os.path.join(ROOT, "train.csv"))
    train_df["target_name"] = train_df["sample_id"].apply(lambda x: x.split("__")[1])

    # Unique images
    unique_train_images = train_df.drop_duplicates(
        subset=["image_path"]
    ).reset_index(drop=True)

    # ---------------------------------------------------------------
    # 3. Extract DINO features (mean pooled)
    # ---------------------------------------------------------------
    embeds = []

    for _, row in tqdm(unique_train_images.iterrows(),
                       total=len(unique_train_images),
                       desc=f"  {desc} Extracting train features"):

        img_path = os.path.join(ROOT, row["image_path"])
        with Image.open(img_path) as img:
            inputs = processor(images=img, return_tensors="pt").to(DEVICE)

        with torch.no_grad():
            out = model(**inputs)
            feat = out.last_hidden_state.mean(dim=1).cpu()
            embeds.append(feat)

    embeds_np = np.array(torch.cat(embeds))     # shape = (N, embed_dim)

    # Standardize embeddings (very important for Lasso)
    scaler = StandardScaler()
    embeds_np = scaler.fit_transform(embeds_np)

    # ---------------------------------------------------------------
    # 4. Build target matrix [N, 5]
    # ---------------------------------------------------------------
    pivot_df = train_df.pivot_table(
        index="image_path",
        columns="target_name",
        values="target",
        aggfunc="mean"
    ).reset_index()

    pivot_df = unique_train_images[["image_path"]].merge(
        pivot_df, on="image_path", how="left"
    )

    columns_order = [
        "Dry_Clover_g",
        "Dry_Dead_g",
        "Dry_Green_g",
        "Dry_Total_g",
        "GDM_g"
    ]

    targets_np = pivot_df[columns_order].values    # shape (N, 5)

    # ---------------------------------------------------------------
    # 5. Train 5 Lasso models × 5 folds
    # ---------------------------------------------------------------
    print("  Training Lasso regression models...")

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    regressors = [[None] * 5 for _ in range(5)]
    oof_preds_np = np.zeros_like(targets_np)

    for t in range(5):     # loop over target index
        y = targets_np[:, t]

        for fold, (tr_idx, val_idx) in enumerate(kf.split(embeds_np)):

            X_tr, y_tr = embeds_np[tr_idx], y[tr_idx]
            X_val, y_val = embeds_np[val_idx], y[val_idx]

            reg = Lasso()
            reg.fit(X_tr, y_tr)

            oof_preds_np[val_idx, t] = reg.predict(X_val)
            regressors[t][fold] = reg

    # Save OOF
    oof_df = pd.DataFrame(oof_preds_np, columns=columns_order)
    oof_df["image_path"] = unique_train_images["image_path"]
    oof_df.to_csv("oof_model1.csv", index=False)

    # ---------------------------------------------------------------
    # 6. Inference on test data
    # ---------------------------------------------------------------
    print("  Running predictions on test data...")

    test_df = pd.read_csv(os.path.join(ROOT, "test.csv"))
    test_embeds = {}

    for img_path in tqdm(test_df["image_path"].unique(),
                         desc=f"  {desc} Extracting test features"):

        full_path = os.path.join(ROOT, img_path)
        with Image.open(full_path) as img:
            inputs = processor(images=img, return_tensors="pt").to(DEVICE)

        with torch.no_grad():
            out = model(**inputs)
            feat = out.last_hidden_state.mean(dim=1).cpu()

        img_name = os.path.splitext(os.path.basename(img_path))[0]
        test_embeds[img_name] = scaler.transform(np.array(feat))

    preds, sample_ids = [], []
    target_mapping = {name: i for i, name in enumerate(columns_order)}

    # Predict per sample_id
    for _, row in test_df.iterrows():
        sample_id = row["sample_id"]
        img_name, target_name = sample_id.split("__")

        t = target_mapping[target_name]
        X_test = test_embeds[img_name]

        fold_preds = [reg.predict(X_test) for reg in regressors[t]]
        pred = np.mean(fold_preds)
        preds.append(max(0, pred))   # clip negatives
        sample_ids.append(sample_id)

    submission = pd.DataFrame({"sample_id": sample_ids, "target": preds})
    return submission.sort_values("sample_id").reset_index(drop=True)


# ---------------------------------------------------------------
# MAIN
# ---------------------------------------------------------------
if __name__ == "__main__":
    print(f"--- [Start] Model 1: DINOv2-Giant + Lasso ---")
    print(f"Inference device: {DEVICE}")

    submission1 = run_dinov2_lasso_inference("giant", "Model 1 (DINOv2-Giant)")
    submission1.to_csv("submission_dino_giant.csv", index=False)

    print("--- [Done] Saved to submission_dino_giant.csv ---")

    del submission1
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


Writing model_1_dino_giant.py


## Model 2 Convnext Tiny

In [2]:
%%writefile model_2_ConvnextTiny.py

from __future__ import annotations
from dataclasses import dataclass, field
from pathlib import Path
from typing import Optional
from collections import OrderedDict
import os
import gc

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import cv2
from tqdm import tqdm


# ============================================================================
# Configuration Management
# ============================================================================

@dataclass
class InferenceConfig:
    """
    Data class for managing inference pipeline configuration.

    The following items must match the training configuration:
    - model_name
    - img_size
    - target column names
    """

    # --- Path settings ---
    base_path: Path = Path('/kaggle/input/csiro-biomass')
    test_csv: Path = field(init=False)
    test_image_dir: Path = field(init=False)
    model_dir: Path = Path('/kaggle/input/csiro-exp3/convnext_exp3') # Directory where trained models are stored
    submission_file: str = 'submission_ConvnextTiny.csv'

    # --- Model settings (must match training) ---
    model_name: str = 'convnext_small' # Backbone model to use
    img_size: int = 1000 # Input image size

    # --- Device settings ---
    device: torch.device = field(default_factory=lambda: torch.device(
        'cuda' if torch.cuda.is_available() else 'cpu'
    ))

    # --- Inference settings ---
    batch_size: int = 1
    num_workers: int = 1
    n_folds: int = 5 # Number of folds for ensemble

    # --- Target settings (must match training) ---
    # The 3 targets the model directly predicts
    train_target_cols: list[str] = field(default_factory=lambda: [
        'Dry_Total_g', 'GDM_g', 'Dry_Green_g'
    ])

    # All 5 targets required for submission
    all_target_cols: list[str] = field(default_factory=lambda: [
        'Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g'
    ])

    def __post_init__(self) -> None:
        """Construct paths after initialization"""
        self.test_csv = self.base_path / 'test.csv'
        self.test_image_dir = self.base_path / 'test'

    def display_info(self) -> None:
        """Display configuration information"""
        print(f"{'='*70}")
        print(f"Inference Configuration")
        print(f"{'='*70}")
        print(f"Device: {self.device}")
        print(f"Backbone: {self.model_name}")
        print(f"Image Size: {self.img_size}x{self.img_size}")
        print(f"Batch Size: {self.batch_size}")
        print(f"Ensemble: {self.n_folds}-Fold")
        print(f"TTA: 3 Views (Original, Horizontal Flip, Vertical Flip)")
        print(f"{'='*70}\n")


# ============================================================================
# TTA (Test-Time Augmentation) Transforms
# ============================================================================

class TTATransformFactory:
    """
    Factory class for generating Test Time Augmentation transforms.

    Provides 3 different views:
    1. Original (no augmentation)
    2. Horizontal flip
    3. Vertical flip
    """

    def __init__(self, img_size: int):
        """
        Args:
            img_size: Image size after resizing
        """
        self.img_size = img_size

        # Base transforms common to all views
        self.base_transforms = [
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Standard ImageNet normalization
            ToTensorV2() # Convert to PyTorch tensor format
        ]

    def get_tta_transforms(self) -> list[A.Compose]:
        """
        Generate 3 transform pipelines for TTA.

        Returns:
            List of 3 Albumentations.Compose objects

        Why not add more TTA variations?
            → Considering the trade-off with inference time.
        """
        # View 1: Original
        original = A.Compose([
            A.Resize(self.img_size, self.img_size),
            *self.base_transforms
        ])

        # View 2: Horizontal flip
        hflip = A.Compose([
            A.HorizontalFlip(p=1.0), # Apply horizontal flip with 100% probability
            A.Resize(self.img_size, self.img_size),
            *self.base_transforms
        ])

        # View 3: Vertical flip
        vflip = A.Compose([
            A.VerticalFlip(p=1.0), # Apply vertical flip with 100% probability
            A.Resize(self.img_size, self.img_size),
            *self.base_transforms
        ])

        return [original, hflip, vflip]


# ============================================================================
# Dataset
# ============================================================================

class TestBiomassDataset(Dataset):
    """
    Two-stream dataset for testing.

    Accepts a specific transform pipeline for TTA and applies
    the same augmentation to both left and right images.

    Returns:
        tuple: (img_left, img_right) (left image tensor, right image tensor)
    """

    def __init__(
        self,
        df: pd.DataFrame,
        transform_pipeline: A.Compose,
        image_dir: Path
    ):
        """
        Args:
            df: DataFrame containing image paths
            transform_pipeline: Augmentation pipeline to apply
            image_dir: Path to the image directory
        """
        self.df = df
        self.transform = transform_pipeline
        self.image_dir = image_dir
        self.image_paths = df['image_path'].values

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Get one sample.

        Args:
            idx: Sample index

        Returns:
            (left_image, right_image): Tuple of left and right image tensors

        Why not apply different augmentations to left/right as in training?
            → During TTA, apply the same transform to both images to preserve symmetry.
        """
        img_path = self.image_paths[idx]
        full_path = self.image_dir / Path(img_path).name

        # Load image (return black image on error)
        image = cv2.imread(str(full_path))

        if image is None:
            print(f"Warning: Failed to load image: {full_path} -> Returning black image")
            image = np.zeros((1000, 2000, 3), dtype=np.uint8)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert from OpenCV (BGR) to RGB

        # Split into left and right
        height, width = image.shape[:2]
        mid_point = width // 2
        img_left = image[:, :mid_point]
        img_right = image[:, mid_point:]

        # Apply same transform to both
        img_left_tensor = self.transform(image=img_left)['image']
        img_right_tensor = self.transform(image=img_right)['image']

        return img_left_tensor, img_right_tensor


# ============================================================================
# Model
# ============================================================================

class BiomassModel(nn.Module):
    """
    Two-stream, three-head regression model.

    Uses the exact same architecture as during training.
    """

    def __init__(self, model_name: str, pretrained: bool = False):
        """
        Args:
            model_name: timm model name
            pretrained: Whether to use pretrained weights (False for inference, as custom weights are loaded later)
        """
        super().__init__()

        # Shared backbone for both streams
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0,       # Classifier layer is not needed
            global_pool='avg'    # Use GAP (Global Average Pooling)
        )

        self.n_features = self.backbone.num_features # Number of output features from the backbone
        self.n_combined = self.n_features * 2        # Number of features after concatenating left and right streams

        # Dedicated prediction heads for each of the three targets
        self.head_total = self._create_head() # Head for Dry_Total_g
        self.head_gdm = self._create_head()   # Head for GDM_g
        self.head_green = self._create_head() # Head for Dry_Green_g

    def _create_head(self) -> nn.Sequential:
        """Helper function to generate the MLP structure for a single head"""
        return nn.Sequential(
            nn.Linear(self.n_combined, self.n_combined // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined // 2, 1),
            nn.ReLU()  # Enforce non-negative biomass outputs
        )


    def forward(
        self,
        img_left: torch.Tensor,
        img_right: torch.Tensor
    ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Forward pass.

        Args:
            img_left: Left image tensor [B, C, H, W]
            img_right: Right image tensor [B, C, H, W]

        Returns:
            (total_pred, gdm_pred, green_pred): Tuple of predictions (each [B, 1])
        """
        feat_left = self.backbone(img_left)   # Extract features from the left image
        feat_right = self.backbone(img_right) # Extract features from the right image
        combined = torch.cat([feat_left, feat_right], dim=1) # Concatenate features

        # Calculate predictions with each head
        out_total = self.head_total(combined)
        out_gdm = self.head_gdm(combined)
        out_green = self.head_green(combined)

        return out_total, out_gdm, out_green


# ============================================================================
# Model Loader
# ============================================================================

class ModelLoader:
    """
    Class for loading trained models.

    Handles weights saved with DataParallel.
    """

    def __init__(self, config: InferenceConfig):
        """
        Args:
            config: Configuration object
        """
        self.config = config

    def load_fold_models(self) -> list[nn.Module]:
        """
        Load all 5-Fold trained models.

        Returns:
            List of models (each in eval mode on the specified device)

        Raises:
            FileNotFoundError: If a model file is not found
        """
        print(f"\nLoading {self.config.n_folds} trained models...")

        models = []

        for fold in range(self.config.n_folds):
            model_path = self.config.model_dir / f'best_model_fold{fold}.pth'

            if not model_path.exists():
                raise FileNotFoundError(f"Model file not found: {model_path}")

            # Initialize model
            model = BiomassModel(self.config.model_name, pretrained=False)

            # Load weights
            state_dict = torch.load(model_path, map_location=self.config.device)

            # Remove 'module.' prefix from DataParallel
            state_dict = self._remove_dataparallel_prefix(state_dict)

            model.load_state_dict(state_dict)
            model.eval()  # Set to evaluation mode
            model.to(self.config.device) # Move model to GPU/CPU

            models.append(model)
            print(f"  ✓ Fold {fold} model loaded")

        print(f"✓ Successfully loaded {len(models)} models\n")
        return models

    @staticmethod
    def _remove_dataparallel_prefix(state_dict: dict) -> dict:
        """
        Remove the 'module.' prefix from keys in a state_dict saved with DataParallel.

        Args:
            state_dict: Model weight dictionary

        Returns:
            Weight dictionary with the prefix removed

        Why not use try-except with a direct load_state_dict call?
            → Explicitly handling the prefix presence improves readability.
        """
        if not any(k.startswith('module.') for k in state_dict.keys()):
            return state_dict  # Return as is if no prefix is found

        # Create a new dictionary with modified keys
        new_state_dict = OrderedDict()
        for key, value in state_dict.items():
            new_key = key.replace('module.', '')
            new_state_dict[new_key] = value

        return new_state_dict


# ============================================================================
# Inference Engine
# ============================================================================

class InferenceEngine:
    """
    Engine for executing TTA + Ensemble inference.
    """

    def __init__(
        self,
        models: list[nn.Module],
        config: InferenceConfig
    ):
        """
        Args:
            models: List of trained models (for 5 folds)
            config: Configuration object
        """
        self.models = models
        self.config = config

    def predict_single_view(
        self,
        loader: DataLoader
    ) -> dict[str, np.ndarray]:
        """
        Predict with 5-Fold Ensemble for one TTA view.

        Args:
            loader: DataLoader (with a specific TTA transform applied)

        Returns:
            Dictionary of predictions in the format {'total': [N], 'gdm': [N], 'green': [N]}
        """
        view_preds = {'total': [], 'gdm': [], 'green': []}

        with torch.no_grad(): # Disable gradient calculation
            for img_left, img_right in tqdm(loader, desc="  Predicting", leave=False):
                img_left = img_left.to(self.config.device)
                img_right = img_right.to(self.config.device)

                # Collect predictions from 5 folds
                fold_preds = {'total': [], 'gdm': [], 'green': []}

                for model in self.models:
                    pred_total, pred_gdm, pred_green = model(img_left, img_right)
                    fold_preds['total'].append(pred_total.cpu())
                    fold_preds['gdm'].append(pred_gdm.cpu())
                    fold_preds['green'].append(pred_green.cpu())

                # Average predictions across 5 folds
                avg_total = torch.mean(torch.stack(fold_preds['total']), dim=0)
                avg_gdm = torch.mean(torch.stack(fold_preds['gdm']), dim=0)
                avg_green = torch.mean(torch.stack(fold_preds['green']), dim=0)

                view_preds['total'].append(avg_total.numpy())
                view_preds['gdm'].append(avg_gdm.numpy())
                view_preds['green'].append(avg_green.numpy())

        # Concatenate results from all batches
        return {
            k: np.concatenate(v).flatten()
            for k, v in view_preds.items()
        }

    def predict_with_tta(
        self,
        test_df: pd.DataFrame,
        tta_transforms: list[A.Compose]
    ) -> dict[str, np.ndarray]:
        """
        Execute final prediction with TTA + Ensemble.

        Args:
            test_df: Test data DataFrame
            tta_transforms: List of transforms for TTA

        Returns:
            Dictionary of final predictions after TTA averaging
        """
        print(f"\nStarting TTA inference: {len(tta_transforms)} Views × {self.config.n_folds} Folds")

        all_view_preds: list[dict[str, np.ndarray]] = []

        for i, transform in enumerate(tta_transforms):
            print(f"--- TTA View {i+1}/{len(tta_transforms)} ---")

            # Create a dedicated Dataset and DataLoader for this view
            dataset = TestBiomassDataset(
                test_df,
                transform,
                self.config.test_image_dir
            )

            loader = DataLoader(
                dataset,
                batch_size=self.config.batch_size,
                shuffle=False,
                num_workers=self.config.num_workers,
                pin_memory=True
            )

            # Perform 5-Fold Ensemble prediction
            view_preds = self.predict_single_view(loader)
            all_view_preds.append(view_preds)

            print(f"  ✓ View {i+1} completed")

        # TTA Ensemble (average across all views)
        print("\nCalculating TTA Ensemble (averaging all views)...")
        final_preds = {
            'total': np.mean([p['total'] for p in all_view_preds], axis=0),
            'gdm': np.mean([p['gdm'] for p in all_view_preds], axis=0),
            'green': np.mean([p['green'] for p in all_view_preds], axis=0)
        }

        print("✓ Inference completed\n")
        return final_preds


# ============================================================================
# Submission Creation
# ============================================================================

class SubmissionCreator:
    """
    Class for creating the Kaggle submission CSV from predictions.
    """

    def __init__(self, config: InferenceConfig):
        """
        Args:
            config: Configuration object
        """
        self.config = config

    def create(
        self,
        predictions: dict[str, np.ndarray],
        test_df_long: pd.DataFrame,
        test_df_unique: pd.DataFrame
    ) -> None:
        """
        Create and save the submission CSV from predictions.

        Args:
            predictions: Predictions in the format {'total': [...], 'gdm': [...], 'green': [...]}
            test_df_long: Original test.csv (long format)
            test_df_unique: DataFrame with only unique images

        Processing flow:
        1. Calculate 5 targets from 3 predictions
        2. Create a wide-format DataFrame
        3. Convert to long format (melt)
        4. Merge with sample_id
        5. Save as CSV
        """
        print("Creating submission CSV...")

        # 1. Get the 3 predictions output by the model
        pred_total = predictions['total']
        pred_gdm = predictions['gdm']
        pred_green = predictions['green']

        # 2. Calculate the remaining 2 targets using relationships (clip negative values to 0)
        pred_clover = np.maximum(0, pred_gdm - pred_green)
        pred_dead = np.maximum(0, pred_total - pred_gdm)

        # 3. Create a wide-format DataFrame
        preds_wide = pd.DataFrame({
            'image_path': test_df_unique['image_path'],
            'Dry_Green_g': pred_green,
            'Dry_Dead_g': pred_dead,
            'Dry_Clover_g': pred_clover,
            'GDM_g': pred_gdm,
            'Dry_Total_g': pred_total
        })

        # 4. Convert to long format (unpivot)
        preds_long = preds_wide.melt(
            id_vars=['image_path'],
            value_vars=self.config.all_target_cols,
            var_name='target_name',
            value_name='target'
        )

        # 5. Merge with the original test.csv to get sample_id
        submission = pd.merge(
            test_df_long[['sample_id', 'image_path', 'target_name']],
            preds_long,
            on=['image_path', 'target_name'],
            how='left'
        )

        # 6. Format and save
        submission = submission[['sample_id', 'target']]
        submission.to_csv(self.config.submission_file, index=False)

        print(f"\n🎉 Submission saved to: {self.config.submission_file}")
        print("\n--- First 5 rows ---")
        print(submission.head())
        print("\n--- Last 5 rows ---")
        print(submission.tail())


# ============================================================================
# Inference Pipeline
# ============================================================================

class InferencePipeline:
    """
    Class that orchestrates the entire inference pipeline.
    """

    def __init__(self, config: InferenceConfig):
        """
        Args:
            config: Configuration object
        """
        self.config = config
        self.model_loader = ModelLoader(config)
        self.tta_factory = TTATransformFactory(config.img_size)
        self.submission_creator = SubmissionCreator(config)

    def run(self) -> None:
        """
        Execute the entire inference pipeline.

        Processing flow:
        1. Load test data
        2. Load models (5-Fold)
        3. Run TTA inference (3 Views × 5 Folds)
        4. Create submission file
        """
        print(f"\n{'='*70}")
        print(f"🚀 Starting Inference Pipeline")
        print(f"{'='*70}")

        try:
            # 1. Load test data
            test_df_long, test_df_unique = self._load_test_data()

            # 2. Load models
            models = self.model_loader.load_fold_models()

            # 3. Run TTA inference
            engine = InferenceEngine(models, self.config)
            tta_transforms = self.tta_factory.get_tta_transforms()
            predictions = engine.predict_with_tta(test_df_unique, tta_transforms)

            # 4. Create submission file
            self.submission_creator.create(
                predictions,
                test_df_long,
                test_df_unique
            )

            print("\n✨ Inference Pipeline Completed Successfully ✨")

        except Exception as e:
            print(f"\n❌ An error occurred: {e}")
            raise

        finally:
            # Free up memory
            del models, engine, predictions
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    def _load_test_data(self) -> tuple[pd.DataFrame, pd.DataFrame]:
        """
        Load test data.

        Returns:
            (test_df_long, test_df_unique)
            - test_df_long: Original long-format DataFrame (with sample_id)
            - test_df_unique: DataFrame filtered to unique images only

        Raises:
            FileNotFoundError: If test.csv is not found
        """
        print(f"\nLoading test data: {self.config.test_csv}")

        if not self.config.test_csv.exists():
            raise FileNotFoundError(f"test.csv not found: {self.config.test_csv}")

        test_df_long = pd.read_csv(self.config.test_csv)
        # Since predictions are made per image, create a DataFrame with duplicate image paths removed
        test_df_unique = test_df_long.drop_duplicates(
            subset=['image_path']
        ).reset_index(drop=True)

        print(f"  Long format data: {len(test_df_long)} rows")
        print(f"  Unique images: {len(test_df_unique)} images\n")

        return test_df_long, test_df_unique


# ============================================================================
# Main Execution Block
# ============================================================================

if __name__ == '__main__':
    # Initialize configuration
    config = InferenceConfig()
    config.display_info()

    # Run the pipeline
    pipeline = InferencePipeline(config)
    pipeline.run()

    print("\n" + "="*70)
    print("🎊 All inference processes have completed!")
    print("="*70)

Writing model_2_ConvnextTiny.py


## Model 3 Google SigLIP

In [3]:
%%writefile model_3_siglip.py
# ==============================================================================
# 0. Library Imports and Initial Setup
# ==============================================================================
import warnings
warnings.filterwarnings('ignore') # Suppress warnings

from pathlib import Path
from tqdm.auto import tqdm
import json
from copy import deepcopy
import polars as pl
import numpy as np
import os

import torch
from PIL import Image
from transformers import AutoProcessor, AutoImageProcessor, AutoModel

from sklearn.model_selection import GroupKFold
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler

import catboost

# --- Initial Setup ---
# Set device to 'cuda' if GPU is available, otherwise 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Path where the data is stored
data_path = Path('/kaggle/input/csiro-biomass')

# List of target variables (types of biomass) to predict
labels = [
  "Dry_Clover_g",
  "Dry_Dead_g",
  "Dry_Green_g",
  "Dry_Total_g",
  "GDM_g"
]

# ==============================================================================
# 1. Feature Preparation
# ==============================================================================

# --- 1.1. Load SigLIP Model ---
print("Loading SigLIP model...")
# Load the pretrained SigLIP model
# SigLIP is a Vision-Language model capable of extracting high-quality feature vectors from images.
model_name = "/kaggle/input/google-siglip-so400m-patch14-384/transformers/default/1/"
model = AutoModel.from_pretrained(
    model_name,
)
model = model.to(device)  # Move the model to the GPU
model.eval()              # Set the model to evaluation mode (disables gradient calculation)

# Load the corresponding image processor (for preprocessing)
processor = AutoImageProcessor.from_pretrained(model_name)
print("Model loading complete.")

# --- 1.2. Load and Preprocess Training Data ---
print("Processing training data...")
train = pl.read_csv(data_path / 'train.csv')
df = (
    train
    # Create columns for each label name based on 'target_name' and store the 'target' value (pivot operation)
    .with_columns([
        pl.when(pl.col('target_name') == label).then(pl.col('target')).alias(label)
        for label in labels
    ])
    # Group by image_path
    .group_by('image_path')
    # Calculate the mean for each label and create a group key for GroupKFold
    .agg([
        pl.col(label).mean()
        for label in labels
    ] + [
        pl.concat_str(["Sampling_Date", "State"], separator=" ").alias("group").first()
    ])
    .sort('image_path') # Sort by image_path
)

# --- 1.3. Load and Preprocess Test Data ---
print("Processing test data...")
test = pl.read_csv(data_path / 'test.csv')
df_test = (
    test
    .group_by('image_path')
    .len() # Get the number of targets for each image (5 in this case)
    .sort('image_path')
)

# --- 1.4. Extract Image Features with SigLIP ---
def compute_features(images: list, save_path: str):
    """Function to take a list of images, compute features with the SigLIP model, and save them to an ndjson file."""
    batch_size = 20
    with torch.no_grad(), open(save_path, 'w') as f:
        for i in tqdm(range(0, len(images), batch_size), desc=f"Extracting {save_path}"):
            batch_paths = images[i:i + batch_size]
            batch = [Image.open(data_path / p) for p in batch_paths]
            inputs = processor(images=batch, return_tensors="pt").to(model.device)
            features = model.get_image_features(**inputs)
            for line in features:
                data = {f'x_{j}': line[j].item() for j in range(len(line))}
                f.write(json.dumps(data) + '\n')

compute_features(df['image_path'], 'features.ndjson')
compute_features(df_test['image_path'], 'features_test.ndjson')
print("Feature extraction complete.")

# --- 1.5. Combine Features with Original Data ---
responses = pl.read_ndjson('features.ndjson')
responses_test = pl.read_ndjson('features_test.ndjson')
df_aug = pl.concat([df, responses], how='horizontal')
df_test_aug = pl.concat([df_test, responses_test], how='horizontal')

# ==============================================================================
# 2. Validation Setup
# ==============================================================================

# --- 2.1. Define Evaluation Metric and Weights ---
weights = {
    'Dry_Green_g': 0.1,
    'Dry_Dead_g': 0.1,
    'Dry_Clover_g': 0.1,
    'GDM_g': 0.2,
    'Dry_Total_g': 0.5,
}

def competition_metric(y_true, y_pred) -> float:
    """Function to calculate the competition's official evaluation metric (weighted R2 score)."""
    weights_array = np.array([weights[l] for l in labels])
    
    # Align with this calculation method
    y_weighted_mean = np.average(y_true, weights=weights_array, axis=1).mean()
    
    # For ss_res and ss_tot, also take the weighted average on axis=1, then the mean of the result
    ss_res = np.average((y_true - y_pred)**2, weights=weights_array, axis=1).mean()
    ss_tot = np.average((y_true - y_weighted_mean)**2, weights=weights_array, axis=1).mean()
    
    return 1 - ss_res / ss_tot

# --- 2.2. Define Cross-Validation Logic ---
def cross_validate(model, data, data_test, x_columns, random_state=42) -> tuple:
    """Function to perform GroupKFold cross-validation with a given model."""
    X = data.select(x_columns).to_numpy()
    X_test = data_test.select(x_columns).to_numpy()
    y_true = data.select(labels).to_numpy()

    y_pred_oof = np.zeros_like(y_true)
    y_pred_test = np.zeros([len(X_test), len(labels)])

    n_splits = 5
    kf = GroupKFold(n_splits=n_splits)
    groups = data.select('group').to_numpy().reshape(-1)

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, groups=groups)):

        # Standardize inside fold (VERY IMPORTANT)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X[train_idx])
        X_val   = scaler.transform(X[val_idx])
        X_test_scaled = scaler.transform(X_test)

        for l in range(len(labels)):
            m = deepcopy(model)
            m.fit(X_train, y_true[train_idx, l])

            y_pred_oof[val_idx, l] = m.predict(X_val).clip(0)
            y_pred_test[:, l] += m.predict(X_test_scaled).clip(0) / n_splits

        score = competition_metric(y_true[val_idx], y_pred_oof[val_idx])
        print(f"Fold {fold}: Score = {score:.6f}")

    full_cv_score = competition_metric(y_true, y_pred_oof)
    print(f"Full CV Score: {full_cv_score:.6f}")

    return y_pred_oof, y_pred_test


# ==============================================================================
# 3. Model Selection & Ensemble
# ==============================================================================

feature_columns = sorted(responses.columns)

# --- 3.1. Compare Performance of Multiple Models ---
print("\n--- [Comparison] DummyRegressor ---")
cross_validate(DummyRegressor(), df_aug, df_test_aug, feature_columns);

print("\n--- [Comparison] Ridge ---")
cross_validate(Ridge(), df_aug, df_test_aug, feature_columns);

print("\n--- [Comparison] Lasso ---")
cross_validate(Lasso(), df_aug, df_test_aug, feature_columns);

# --- [Final Model] GradientBoostingRegressor ---
print("\n--- [Final Model] GradientBoostingRegressor ---")
oof_pred_gb, pred_test_gb = cross_validate(GradientBoostingRegressor(random_state=42), df_aug, df_test_aug, feature_columns)

# --- [Final Model] CatBoostRegressor ---
print("\n--- [Final Model] CatBoostRegressor ---")
oof_pred_cb, pred_test_cb = cross_validate(catboost.CatBoostRegressor(verbose=False, iterations=100, random_state=42), df_aug, df_test_aug, feature_columns)

# ★★★★★ Ensemble and Save OOF Predictions ★★★★★
print("\nEnsembling and saving OOF predictions...")
oof_pred_ensemble = (oof_pred_gb + oof_pred_cb) / 2
oof_df = df_aug.select(['image_path']).to_pandas()
oof_df[labels] = oof_pred_ensemble
oof_df.to_csv('oof_model3.csv', index=False)

# --- 3.2. Ensemble Model Predictions ---
print("\nEnsembling the predictions of the two models...")
pred_test = (pred_test_gb + pred_test_cb) / 2

# ==============================================================================
# 4. Create Submission File
# ==============================================================================

# --- Convert prediction results to a DataFrame ---
pred_with_id = pl.concat([
    df_test.select("image_path"),
    pl.DataFrame(pred_test, schema=labels),
], how='horizontal')

# --- Format for submission ---
pred_save = (
    test
    .join(pred_with_id, on='image_path')
    .with_columns(
        pl.coalesce(*[  
            pl.when(pl.col('target_name') == col).then(pl.col(col))
            for col in labels
        ]).alias('target')
    )
    .select('sample_id', 'target')
)

# --- Save as CSV file ---
pred_save.write_csv('submission_SigLIP.csv')
print("\nCreated submission_SigLIP.csv.")
print("Partial submission file:")
print(pred_save.head())

Writing model_3_siglip.py


In [4]:
# import time 

!python /kaggle/working/model_1_dino_giant.py
# time.sleep(10)
!python /kaggle/working/model_2_ConvnextTiny.py
# time.sleep(10)
!python /kaggle/working/model_3_siglip.py

2025-11-20 06:33:17.276955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763620397.472821      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763620397.525769      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
--- [Start] Model 1: DINOv2-Giant + Lasso ---
Inference device: cuda
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
  Models loaded.
  Model 1 (DINOv2-Giant) Ex

## Weighted Average Ensemble

In [5]:
# ====================================================================================
# Script to ensemble the prediction results of all models
# ====================================================================================

import os
import pandas as pd

def ensemble():
    print("--- [Final Step] Performing a weighted average of all model predictions ---")
    
    files = {
        "DINOv2-Giant": "/kaggle/working/submission_dino_giant.csv", 
        "ConvnextTiny": "/kaggle/working/submission_ConvnextTiny.csv", 
        "SigLIP": "/kaggle/working/submission_SigLIP.csv"
    }

    # Check if all necessary files exist
    for key, filename in files.items():
        if not os.path.exists(filename):
            print(f"Error: {filename} not found.")
            print("Please run all inference scripts before ensembling.")
            return

    # Load the submission file from each model
    # Map to df1-df3 variables and apply weights
    df1 = pd.read_csv(files["DINOv2-Giant"]).set_index("sample_id")
    df2 = pd.read_csv(files["ConvnextTiny"]).set_index("sample_id") 
    df3 = pd.read_csv(files["SigLIP"]).set_index("sample_id") 
    
    # --- Perform the weighted average ---
    final_submission = (
        0.25 * df1 +   # DINOv2-Giant
        0.40 * df2 +   # ConvnextTiny (LB 0.61)
        0.35 * df3     # SigLip (LB 0.59)
    )    
    
    # Save the final submission file
    final_submission.to_csv("submission.csv")

    print("\n🎉 All processes are complete! The final submission file 'submission.csv' has been created.")
    print("--- First 5 rows of the submission file ---")
    print(final_submission.head())


if __name__ == "__main__":
    ensemble()

--- [Final Step] Performing a weighted average of all model predictions ---

🎉 All processes are complete! The final submission file 'submission.csv' has been created.
--- First 5 rows of the submission file ---
                               target
sample_id                            
ID1001187975__Dry_Clover_g   1.381992
ID1001187975__Dry_Dead_g    24.958363
ID1001187975__Dry_Green_g   33.251118
ID1001187975__Dry_Total_g   57.972596
ID1001187975__GDM_g         33.084863
